In [801]:
import numpy as np
from typing import Callable, List, Tuple, Union

# Solutions of Equations in One Variable

## Chapter 2.1 The Bisection Method
This method is derived by using _Extreme Value Theorem_ as indicated below

### Extreme Value Theorem
If $f\in C[a, b]$, then $c_1, c_2\in[a, b]$ exist with $f(c_1)\leq f(x)\leq f(c_2)$, for all $x\in[a, b]$. In addition, if $f$ is differentiable on $(a, b)$, then the numbers $c_1$ and $c_2$ occur either at the endpoints of $[a, b]$ or where $f'$ is zero.

Given $a, b$ WLOG let $f(a) < 0,f(b) > 0$, then by the _Extreme Value Theorem_ $\exists x\in[a, b]$ s.t. $f(a) < f(x) = 0 < f(b)$ since $f(a)<0<f(b)$. This means that $f(a)f(b) < 0 \implies\exists x\in[a, b]\text{ s.t. }f(x)=0$

In [802]:
def bisection(func:Callable[[float], float], left:float, right:float, iterations:int=30, tol:float=1e-5)->List[float]:
    if func(left) * func(right) > 0:
        raise ValueError("Cannot apply Intermidiate Value Theorem")
    prev_p = np.inf 
    p_ = []
    for _ in range(iterations):
        p = left + (right - left) / 2
        p_.append(p)
        if abs(prev_p - p) / p < tol:
            return p_
        if func(left) * func(p) < 0:
            right = p
        elif func(p) * func(right) < 0:    
            left = p
    return p_

## Chapter 2.2 Fixed-Point Iteration (FPI)
$$g(p)=p$$

In [803]:
def FPI(func:Callable[[float], float], init:float, iterations:int=30, tol:float=1e-5)->List[float]:
    prev_p = np.inf
    p_ = [init]
    p = init
    for _ in range(iterations):
        prev_p = p
        p = func(p)
        p_.append(p)
        if abs(prev_p - p) / p < tol:
            return p_

## Chapter 2.3 Newton's Method and its Extensions
### Newton's Method
$$p_n=p_{n-1}-\frac{f(p_{n-1})}{f'(p_{n-1})}$$

In [804]:
def newton(f:Callable[[float], float], df:Callable[[float], float], init:float, iterations:int=30, tol:float=1e-5)->List[float]:
    prev_p = np.inf
    p_ = [init]
    p = init
    for _ in range(iterations):
        prev_p = p
        p -= f(p) / df(p)
        p_.append(p)
        if abs(prev_p - p) / p < tol:
            return p_

### Secant Method
$$p_n=p_{n-1}-\frac{f(p_{n-1})(p_{n-1}-p_{n-2})}{f(p_{n-1})-f(p_{n-2})}$$

In [805]:
def secant(f:Callable[[float], float], p0:float, p1:float, iterations:int=30, tol:float=1e-5)->List[float]:
    prev_p = np.inf
    p_ = [p0, p1]
    p = p1
    for _ in range(iterations):
        prev_p = p
        p = p1 - (f(p1) * (p1 - p0) / (f(p1) - f(p0)))
        p_.append(p)
        if abs(prev_p - p) / p < tol:
            return p_
        p0 = p1
        p1 = p
    return p_

### False Position
The method of _False Position_ generates approximations
in the same manner as the Secant method, but it includes a test to ensure that the root is
always bracketed between successive iterations (_Extreme Value Theorem_). Although it is not a method we generally
recommend, it illustrates how bracketing can be incorporated.

In [806]:
def falsePosition(f:Callable[[float], float], p0:float, p1:float, iterations:int=30, tol:float=1e-5)->List[float]:
    prev_p = np.inf
    p_ = [p0, p1]
    p = p1
    for _ in range(iterations):
        prev_p = p
        p = p1 - (f(p1) * (p1 - p0) / (f(p1) - f(p0)))
        p_.append(p)
        if abs(prev_p - p) / p < tol:
            return p_
        if f(p) * f(p1) < 0:
            p0 = p1
        p1 = p
    return p_

In [807]:
import math

f = lambda x: x - math.cos(x)
g = lambda x: 1 + math.sin(x)

print(falsePosition(f, 0.5, math.pi / 4, iterations=10))
print(secant(f, 0.5, math.pi / 4, iterations=10))
print(newton(f, g, math.pi / 4, iterations=10))

[0.5, 0.7853981633974483, 0.7363841388365822, 0.7390581392138897, 0.7390848638147098, 0.7390851305265789]
[0.5, 0.7853981633974483, 0.7363841388365822, 0.7390581392138897, 0.7390851493372764, 0.7390851332150645]
[0.7853981633974483, 0.7395361335152383, 0.7390851781060102, 0.739085133215161]


## Chapter 2.6 Zeros of Polynomials and Muller's Method
### Horner's Method
Let
$$ P(x) = a_nx^n+a_{n-1}+x^{n-1}+\cdots+a_1x+a_0 $$
Define $b_n=a_n$ and
$$ b_k=a_k+b_{k+1}x_0,\,\,\,\text{for }k=n-1, n-2\cdots,1, 0$$
Then $b_0=P(x_0)$.Moreover, if
$$ Q(x)=b_nx^{n-1}+b_{n-1}x^{n-2}+\cdots+b_2x+b_1$$
Then 
$$ P(x) = (x-x_0)Q(x)+b_0 $$
Cooperate with _Newton's Method_ to find the zeros of polynomials.

In [808]:
def Horner(coef:Union[np.ndarray, List[Union[int, float]]], 
           x0:Union[float, int]) -> List[Union[float, int, np.ndarray]]:
    coef_ = np.copy(coef).astype(float)
    f_ = np.zeros(len(coef_) - 1, dtype=float)
    f = coef_[-1]
    fp = coef_[-1]
    f_[-1] = coef_[-1]
    for jdx in range(len(coef_) - 2, 0, -1):
        f = x0 * f + coef_[jdx]
        f_[jdx - 1] = f
        fp = x0 * fp + f
    f = x0 * f + coef_[0]
    # f_[0] = f
    return [f, fp, f_]

In [809]:
Horner([-4, 3, -3, 0, 2], -2)

[10.0, -49.0, array([-7.,  5., -4.,  2.])]

In [810]:
import cmath
def hornerNewton(coef:Union[np.ndarray, List[Union[int, float]]], 
                 x0:Union[float, int],
                 iterations:int=100,
                 tol:float=1e-6) -> List[Union[float, int]]:
    p = x0
    roots = []
    coef_ = np.copy(coef).astype(float)
    for _ in range(len(coef) - 1):
        if len(coef_) == 3:
            break
        # Newton's Method to approximate zeros
        prev_p = np.inf
        for _ in range(iterations):
            prev_p = p
            f, fp, _ = Horner(coef_, p)
            p -= f / fp
            if np.abs(prev_p - p) < tol:
                break
        # Horner's Method to divide the original polynomial
        # by the approximated zero
        f, fp, coef_ = Horner(coef_, p)
        roots.append(p)
    r1 = (-coef_[1] + cmath.sqrt(coef_[1] ** 2 - 4 * coef_[0] * coef_[2])) / (2 * coef_[2])
    r2 = (-coef_[1] - cmath.sqrt(coef_[1] ** 2 - 4 * coef_[0] * coef_[2])) / (2 * coef_[2])
    roots.extend([r1, r2])
    return roots

In [811]:
hornerNewton([-5040, 1602, 1127, -214, -72, 4, 1], 8)

[7.0,
 3.0000000000000004,
 2.0000000000000013,
 -3.0000000000000107,
 (-4.999999999999992+0j),
 (-7.999999999999997+0j)]

In [812]:
hornerNewton([-4, 3, -3, 0, 2], -2)

[-1.738956256451892,
 1.2548818848342944,
 (0.24203718580879874+0.926245487267532j),
 (0.24203718580879874-0.926245487267532j)]

### Muller's Method
Müller’s method is similar to the
Secant method. But whereas the
Secant method uses a line
through two points on the curve
to approximate the root, Müller’s
method uses a parabola through three points on the curve for the approximation.

In [813]:
import cmath
Muller_Params = Union[complex, float]
def Muller(f:Callable[[Muller_Params], float], 
           p0:Muller_Params, p1:Muller_Params, p2:Muller_Params, 
           iterations:int=20, tol:float=1e-5) -> Muller_Params:
    for _ in range(iterations):
        h1 = p1 - p0
        h2 = p2 - p1
        delta1 = (f(p1) - f(p0)) / h1
        delta2 = (f(p2) - f(p1)) / h2
        d = (delta2 - delta1) / (h2 + h1)
        b = delta2 + h2 * d
        D = cmath.sqrt(b ** 2 - 4 * f(p2) * d)
        denoms = [b + D, b - D]
        if np.abs(h := -2 * f(p2)) < tol:
            return p
        p = p2 + h / max(denoms, key=abs)
        p0, p1, p2 = p1, p2, p
    return p

In [814]:
f = lambda x: x ** 4 - 3 * x ** 3 + x ** 2 + x + 1
print(Muller(f, 0.5, -0.5, 0))
print(Muller(f, 0.5, 1, 1.5))
print(Muller(f, 1.5, 2, 2.5))

(-0.33909283338402885+0.44663010055724217j)
(1.389389619617081+0j)
(2.288794993947653+0j)
